James Gardner, 2022 

want to analyse science case/s for CE only:

CE-N 40km with CE-S 40km or 20km

if done, then look at CE-S with one ET detector

verify techniques by replicating *Borhanian and Sathya 2022*

In [ ]:
from detection_rates import *

# suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

### Replicating Borhanian and Sathya 2022 injections and detection rates, then for CE only 

In [ ]:
# batch detection rate results for given science case and network set

# waveform, LAL list: https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__h.html
# --- BNS ---
# science_case = 'BNS'
# wf_model_name, wf_other_var_dic = 'lal_bns', dict(approximant='IMRPhenomD_NRTidalv2') # for tidal, see https://arxiv.org/abs/1905.06011
# to-do: missing dimensionless tidal parameters, calculate tidal parameters from sampled m1, m2 in injections.py? requires Love number and radii (i.e. choose an EoS)
# --- BBH ---
science_case = 'BBH'
wf_model_name, wf_other_var_dic = 'lal_bbh', dict(approximant='IMRPhenomHM')
# --- analytic waveforms ---
# wf_model_name, wf_other_var_dic = 'tf2', None # to-do: stop using this once tidal params found
# wf_model_name, wf_other_var_dic = 'tf2_tidal', None

# number of injections per redshift bin (6 bins)
num_injs = 10

# network_spec_list = BS2022_STANDARD_6['nets']
# network_spec_list = CE_ONLY['nets']
network_spec_list = CE_S_W_ET['nets']

for network_spec in tqdm(network_spec_list):
    detection_rate_for_network_and_waveform(network_spec, science_case, wf_model_name, wf_other_var_dic, num_injs, show_fig=False, print_progress=False, print_reach=False, hack_merger_rate_coeff='default')

In [ ]:
# meta-batch to save all the required results (but not generate plots to save time) for the below cell
network_spec_list = BS2022_STANDARD_6['nets'] + CE_ONLY['nets'] + CE_S_W_ET['nets']

science_cases = ('BNS', 'BBH')
# --- numerical BNS as in B&S2021 ---
# wf_specifications = (('lal_bns', dict(approximant='IMRPhenomD_NRTidalv2')), ('lal_bbh', dict(approximant='IMRPhenomHM')))
# --- analytic BNS ---
wf_specifications = (('tf2_tidal', None), ('lal_bbh', dict(approximant='IMRPhenomHM')))

# number of injections per redshift bin (6 bins), tuple over science cases (more for symbolic wf.'s than numerical)
num_injs_list = (100, 10)

for network_spec in tqdm(network_spec_list):
    for j, science_case in enumerate(science_cases):
        wf_model_name, wf_other_var_dic = wf_specifications[j]        
        detection_rate_for_network_and_waveform(network_spec, science_case,
                                                wf_model_name, wf_other_var_dic, num_injs_list[j],
                                                generate_fig=False, show_fig=False,
                                                print_progress=False, print_reach=False,
                                                hack_merger_rate_coeff='default')

### Collating different networks saved using the above method

In [ ]:
# batch of six collated plots: all three network sets and two science cases
# assumes results saved: above results generating cell must be run to save results for this cell

# ! remember to update plot label for each case else the plot will be overwritten
# how standard the "standard 6" are is unclear, seems like an invention of B&S2022
# CE only, with 2G+ as a reference
# CE South with one ET detector

network_sets = (BS2022_STANDARD_6['nets'],
                CE_ONLY['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I'],],
                CE_S_W_ET['nets'] + [['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I'],])
network_labels = ('standard_6', 'CE_only', 'CE_S..ET_ET1')
science_cases = ('BNS', 'BBH')
specific_wfs = ('tf2_tidal', None)

for i, network_set in enumerate(network_sets):
    for j, science_case in enumerate(science_cases):
        plot_label = f'SCI-CASE_{science_case}_NET_{network_labels[i]}'
        specific_wf = specific_wfs[j]
        if specific_wf is not None:
            plot_label += f'_WF_{specific_wf}'
        compare_networks_from_saved_results(network_set, science_case, plot_label=plot_label,
                                            specific_wf=specific_wf,
                                            show_fig=False,
                                            hack_merger_rate_coeff='default') 

### Measurement errors

In [ ]:
def add_measurement_errs_CDFs_to_axs(axs, results_reordered, num_bins, colour, label):
    """add PDFs wrt dlog(x) and CDFs on log-log scale to axs"""
    for i, data in enumerate(results_reordered):
        data.sort() # sorts in place
        log_bins = np.geomspace(data.min(), data.max(), num_bins)
        # density vs weights is normalising integral (area under the curve) vs total count, integral behaves counter-intuitively visually with logarithmic axis 
#         label = legend_label if i==0 else None
        axs[0, i].hist(data, weights=np.full(len(data), 1/len(data)), histtype='step', bins=log_bins, color=colour, label=label)

        # # binned CDF
        # axs[1, i].hist(data, density=True, cumulative=True, histtype='step', color='k', bins=log_bins)
        # axs[1, i].hist(data, density=True, cumulative=-1, histtype='step', color='r', bins=log_bins)
        # unbinned CDF
        cdf = np.arange(len(data))/len(data)
        axs[1, i].plot(data, cdf, color=colour, label=label, zorder=2)
        if i == 0:
            # invert SNR CDF to ``highlight behaviour at large values'' - B&S2022
            axs[1, i].plot(data, 1-cdf, linestyle='--', color=colour, label=label)

def collate_measurement_errs_CDFs_of_networks(network_spec_list, science_case, specific_wf=None, num_bins=20, save_fig=True, show_fig=True, plot_label=None, full_legend=False, print_progress=True):
    """collate PDFs-dlog(x) and CDFs of SNR, sky-area, and measurement errs for given networks"""
    found_files = find_files_given_networks(network_spec_list, science_case, specific_wf=specific_wf, print_progress=print_progress)
    net_labels = [network.Network(network_spec).label for network_spec in network_spec_list]
    if plot_label is None:
        plot_label = ''.join(tuple('_NET_'+l for l in net_labels))[1:]

    plt.rcParams.update({'font.size': 14})
    fig, axs = plt.subplots(2, 6, sharex='col', sharey='row', figsize=(16, 4), gridspec_kw=dict(wspace=0.05, hspace=None))
    
    # same colours manipulation as compare_networks_from_saved_results
    colours_used = []
    for i, file in enumerate(found_files):
        # redshift (z), integrated SNR (rho), measurement errors (logMc, logDL, eta, iota), 90% credible sky area
        # errs: fractional chirp mass, fractional luminosity distance, symmetric mass ratio, inclination angle
        results = np.load(f'data_redshift_snr_errs_sky-area/{file}')
                
        if full_legend:
            legend_label = file_name_to_multiline_readable(file, two_rows_only=True)
        else:
            legend_label = file_name_to_multiline_readable(file, net_only=True)
            
        net_spec = file.replace('NET_', '_SCI-CASE_').split('_SCI-CASE_')[1].split('..')

        if repr(net_spec) in DICT_KEY_NETSPEC_VAL_COLOUR.keys():
            colour = DICT_KEY_NETSPEC_VAL_COLOUR[repr(net_spec)]
            # avoid duplicating colours in plot
            if colour in colours_used:
                colour = None
            else:
                colours_used.append(colour)
        else:
            colour = None        
        
        # re-order results columns to have sky-area second
        results_reordered = (results.transpose()[i] for i in (1, -1, *range(2, 6)))
        add_measurement_errs_CDFs_to_axs(axs, results_reordered, num_bins, colour, legend_label) 
        
#     quantity_long_labels = (r'integrated SNR, $\rho$', r'90%-credible sky area, $\Omega$ / $\mathrm{deg}^2$', r'fractional chirp mass error, $\Delta\mathcal{M}/\mathcal{M}$', r'fractional luminosity distance error, $\Delta D_L/D_L$', r'symmetric mass ratio error, $\Delta\eta$', r'inclination angle error, $\Delta\iota$')
    quantity_short_labels = (r'SNR, $\rho$', r'$\Omega$ / $\mathrm{deg}^2$', r'$\Delta\mathcal{M}/\mathcal{M}$', r'$\Delta D_L/D_L$', r'$\Delta\eta$', r'$\Delta\iota$')

    for i in range(len(axs[1])):
        axs[1, i].axhline(1, color='lightgrey', zorder=1)
        axs[1, i].set(xscale='log', yscale='log', xlabel=quantity_short_labels[i])

    # hist handles are boxes, want lines and so borrow from 1, 1 to avoid dotted
    handles, _ = axs[1, 1].get_legend_handles_labels()
    axs[0, 0].legend(handles=handles, handlelength=1, bbox_to_anchor=(0, -1.7), loc="upper left")
    fig.suptitle(r'collated PDFs wrt $\mathrm{d}\log(x)$ and CDFs'+f'\n{plot_label}', y=0.9, verticalalignment='bottom')
    axs[0, 0].set(ylabel='count')
    axs[1, 0].set(ylabel='CDF', ylim=(1e-4, 1e0+1))
    axs[1, 0].legend(labels=['CDF', '1-CDF'], handles=[mlines.Line2D([], [], color='k'), mlines.Line2D([], [], color='k', linestyle='--')], handlelength=1)
    fig.align_labels()

    if save_fig:
        fig.savefig(f'plots/collated_CDFs_snr_errs_sky-area_{plot_label}.pdf')
    if show_fig:
        plt.show()
    plt.close(fig)

In [ ]:
network_set = BS2022_STANDARD_6['nets']
network_set_label = 'standard_6'
science_case = 'BNS'
specific_wf = 'tf2_tidal'

plot_label = f'SCI-CASE_{science_case}_NETS_{network_set_label}'
if specific_wf is not None:
    plot_label += f'_WF_{specific_wf}'

collate_measurement_errs_CDFs_of_networks(network_set, science_case, specific_wf=specific_wf, plot_label=plot_label, full_legend=False, print_progress=True)

In [ ]:
# B&S2022 plots cumulative density functions (CDF) versus redshift to show distribution of measurement errors (why not just use PDF, i.e. normalised histograms, like B2021?)
# for non-SNR quantities only use results above an SNR threshold (e.g. 10), alternatively, for all non--sky area quantities only use results below a 90% credible sky area threshold (e.g. 4.4e-2 sqrDeg, where the full Moon is ~20.0e-2 sqrDeg given a radius of 0.5 deg)

# ax.hist(data, histtype='step', facecolor='b', label='tf2_tidal', bins=np.geomspace(data.min(), data.max(), 50))

In [ ]:
# to-do: fix merger rate --- find proportionality constant --- to achieve 1e5 per year at z=1 (BBH) z=3 (BNS)
# i.e. stop using hack merger rate coefficients

In [ ]:
# to-do: stop CE alone being ill-conditioned, seems to work for BBH numeric wf

In [ ]:
# to-do: fix BNS numerical waveform, current error "numpy.linalg.LinAlgError: Array must not contain infs or NaNs" with HLVKI+

In [ ]:
# to-do: change sigmoid fits to global optimisation to avoid needing the initial guesses in B&S2022, this is not possible with scipy, need to write my own or source a global curve fitting (apparantly a non-trivial problem)

In [ ]:
# to-do: refactor two plotting functions 

In [ ]:
# to-do: fix BBH for large num_injs, error with frequency array f being too small for M large? --> decrease df 
# gets stuck towards end of progress bar consistently
# IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
# to-do: increase sampling below 4e-2, but App A uses (0, 0.5, seed)?

In [ ]:
# to-do: tidy up refactoring, reduce total number of arguments with unpacking?

In [ ]:
"""In fact, we see that the three generations (A+, Voyager, and NG) are qualitatively different
with respect to every metric used in this study."""